<h2> Hi guys, i want to share the ways to make additional preprocessed tfrecords. </h2>
<h4> (tfrecords is binary form, improving performance in data processing)</h4>
<h2> In this case, i applied band filter just before making tfrecords. </h2>
 
<h2> in this way, you guys also appling every possible techniques. </h2>
<h3> If you find already maded preprocessed tfrecords check it below </h3>
<h4> (https://www.kaggle.com/leemop/using-preprocessed-bandfilter-tfrecords-with-all) </h4>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.python as tfp
from tqdm import tqdm

train_df = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
test_df = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def get_test_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

train_df['image_path'] = train_df['id'].apply(get_train_file_path)
test_df['image_path'] = test_df['id'].apply(get_test_file_path)

def _bytes_feature(value):
    if isinstance(value, tfp.framework.ops.EagerTensor):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

from scipy.signal import butter, lfilter, lfilter_zi, filtfilt
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

LOWCUT = 20
HIGHCUT = 1000

def create_tf_example(wave_id: str, wave: bytes, target: int) -> tf.train.Example:
    feature = {
        "wave_id": _bytes_feature(wave_id),
        "wave": _bytes_feature(wave),
        "target": _float_feature(target)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def write_tfrecord(df: pd.DataFrame, filename: str):    
    options = tf.io.TFRecordOptions("GZIP")
    with tf.io.TFRecordWriter(filename, options=options) as writer:
        for i in tqdm(range(len(df))):
            wave_id = str.encode(df.iloc[i]["id"])
            wave_dir = df.iloc[i]["image_path"]
            wave = butter_bandpass_filter(np.load(wave_dir), LOWCUT, HIGHCUT, 2048, order=7).tobytes()
            #wave = np.load(wave_dir).tobytes()
            target = df.iloc[i]["target"]
            tf_example = create_tf_example(wave_id, wave, target)
            writer.write(tf_example.SerializeToString())    

<h3> One notebook has limited capacity for dataset </h3>
<h3> So, we can seperate into 5 groups. [0,5], [5,10], [10,15], [15,20] </h3>

In [ ]:
#train_samples_per_file = 28000
test_samples_per_file = 22600
test_number_of_files = len(test_df) // test_samples_per_file

for i in range(0,5): 
    start = i * test_samples_per_file
    end = (i + 1) * test_samples_per_file
    df = test_df.iloc[start:end].reset_index(drop=True)
    filename = f"bf_test{i}.tfrecords"                                                                           
    write_tfrecord(df, filename)

<h1> If it is helped please upvote for me :) </h1>
<h1>    Thanks </h1>